In [2]:
from bs4 import BeautifulSoup
import requests

In [49]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime

### FOR INDIVIDUAL LINKS

### web_link of shine.com filtered with trivandrum location

In [4]:
list1=["https://www.shine.com/job-search/jobs-in-thiruvananthapuram-","21","?loc=thiruvananthapuram&location=315"]
url="".join(list1)

In [91]:
url

'https://www.shine.com/job-search/jobs-in-thiruvananthapuram-40?loc=thiruvananthapuram&location=315'

In [1]:
def get_html_file(url):    
    source=requests.get(url).text
    soup=BeautifulSoup(source,'lxml')
    return soup

In [92]:
print(get_html_file(url).prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   Result page 40 for Latest Jobs in Thiruvananthapuram, Kerala (Nov 2020) - 1174 Job Available In Thiruvananthapuram, Kerala - Shine.com
  </title>
  <link href="/static/manifest.json" rel="manifest"/>
  <meta content="profile email" name="google-signin-scope"/>
  <meta content="284765467291-lo164ec1u4pcnhtrbicird7tsdt553sm.apps.googleusercontent.com" name="google-signin-client_id"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="https://static1.shine.com/c/s1/images/candidate/new/NewShinelogo.gif" property="og:image"/>
  <meta content="610312401" property="fb:admins"/>
  <meta content="company" property="og:type"/>
  <meta content="https://static1.shine.com/c/s1/images/candidate/new/NewShinelogo.gif" itemprop="image"/>
  <link href="https://www.shine.com/search.xml

### getting the job specific link to scrap data more efficiently

In [107]:
get_html_file(url).find('div',class_="w-90 ml-25").meta.get("content")

'https://www.shine.com/jobs/customer-service/faya/10269989/'

### function for scrapping the required data from Shine website

In [69]:
def scrapping_data(url):
    source_scrapping=requests.get(url).text
    soup_scrapping=BeautifulSoup(source_scrapping,'lxml')
    article=soup_scrapping.find('article')
# if statement is used because some of the scrapping data like "salary" might not be available for all the job entries 
#and hence they will return None for the particular script and might run an error if the functions are ran on "None data types"
    
    if article.h2.text==None:
        company_name="NA"
    else:
        company_name=article.h2.text
        
    if article.h1.text==None:
        job_role="NA"
    else:
        job_role=article.h1.text
        
    if article.find('li',class_="cls_jobsalary jd_highlight_salary")==None:
        salary="NA"
    elif article.find('li',class_="cls_jobsalary jd_highlight_salary").i.span.text ==None:
        salary="NA"
    else:
        salary=article.find('li',class_="cls_jobsalary jd_highlight_salary").i.span.text
    
    if article.find("div",class_="jobdescriptioninside").span.pre.text==None:
        jd="NA"
    else:
        jd=article.find("div",class_="jobdescriptioninside").span.pre.text
        
    if article.find('div',class_="sal_fun_ind").i==None:
        department="NA"
    elif article.find('div',class_="sal_fun_ind").i.find('span',class_="socialjdsnippet").text==None:
        department="NA"
    else:
        department=article.find('div',class_="sal_fun_ind").i.find('span',class_="socialjdsnippet").text
     
    if article.i.span.text==None:
        req_exp="NA"
    else:
        req_exp=article.i.span.text
# some of the job might be posted just hours ago and i wont be able to convert that into datetime object, hence returns cur_date.
    if article.find("span",class_="jobDate").text==None:
        posted_date="NA"
    else:
        for i in article.find("span",class_="jobDate").stripped_strings:
            try:
                posted_date=pd.to_datetime(i.split(":")[1]).date()
            except TypeError and ValueError:
                posted_date=datetime.today().date()
                 
    
    return company_name,job_role,salary,jd,department,req_exp,posted_date

### taking all the necessary data individually

In [108]:
url_job=get_html_file(url).find('div',class_="w-90 ml-25").meta.get("content")

In [109]:
scrapping_data(url_job)

('FAYA',
 'Customer Service',
 'NA',
 '\nCustomer Service \nCustomer Service Associate \nTrivandrum, India \nApply Now,',
 'Sales / BD',
 '2 to 7 Yrs',
 datetime.date(2020, 10, 19))

### scrapping the data from all the job postings

In [117]:
df=pd.DataFrame(columns=["Company Name","Job Role","Salary","Job Decription","Department","Required Exp","posted_date"])

In [115]:
web_list=["https://www.shine.com/job-search/jobs-in-thiruvananthapuram-","1","?loc=thiruvananthapuram&location=315"]
j=0

In [118]:
# 59 is the page number in the shine.com. The for loop is to run through all the pages. 
for i in range (1,59):
    web_list[1]=str(i)
    soup=get_html_file("".join(web_list))
    ### for looping through all the job posts in a single page
    link=soup.find('div',{"class":'force-overflow base_color pl-10 pt-10 pr-10'}).ul
    for k in link.find_all('li'):
        if k.find('div',{"class":'w-90 ml-25'}) != None:
            web_link=k.find('div',{"class":'w-90 ml-25'}).meta.get("content")
            df.loc[j]=scrapping_data(str(web_link))
            j+=1
            print(j,end=" ")
        else:
            continue
    

1 2 3 4 5 6 7 8 9 10 

KeyboardInterrupt: 

In [103]:
df=pd.concat([df1,df2],axis=1)

In [111]:
df["site"]="Shine.com"

In [113]:
df.to_csv("D:\\reach_technologies_internship\\job_listing_data_Shine.csv")